In [1]:
import pickle as pkl
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler

from utils_camera import MyRealSense, CVCamera
import cv2
from matplotlib import pyplot as plt

import os
import json
import h5py
import numpy as np
import imageio

In [2]:
class ROSInterface:
    def __init__(self, node, robot):
        
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

        self.latest_joint_states = None

    def joint_states_callback(self, msg):
        self.latest_joint_states = msg
        q = self.robot.getJoints()
        # print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10) 
        rclpy.spin(self.node)

In [3]:
rclpy.init()
robot = URDFModel('/home/ns/sawyer_robot_ros2/src/teleop_script/sawyer.urdf')
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}

node = Node('pub_dbg')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)

t1 = Thread(target=ros_interface.spin_thread)
t1.start()

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [4]:
dataset_path = "/home/ns/data_robomimic/drawer/drawer_push86v3.h5" 

In [5]:
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())

In [6]:
lengths=[]
for demo_name in demos:
    demo=f['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  86
Max length:  234
Min length:  14
Mean length:  59.22093023255814


In [7]:
demo_name="demo_12"
demo=f['data'][demo_name]
demo['obs'].keys()

jpos=demo['obs']['robot0_joint_pos']
jvel=demo['obs']['robot0_joint_vel']
dxyz=demo['actions']

jpos.shape

(80, 7)

In [8]:
def get_ee(jointmsg):
    msg=jointmsg    
    q =  robot.getJoints() 
    for ind, name in enumerate(msg.name):
        if name in jointMap:
            q[jointMap[name]] = msg.position[ind]
    robot.setJoints(q)
    
    # T=robot.getBodyTransform('right_l6');
    T=robot.getBodyTransform('camera_link');
    xyz=T[0:3,3]
    e1 = rotm2euler.rotationMatrixToEulerAngles(T[0:3, 0:3])
    ee6=np.concatenate([xyz, e1])
    return ee6
def goto_xyzrpy(target, delay=0.1):
    ee6=get_ee(ros_interface.latest_joint_states)
    xd=ee6-target

    alpha = .0001
    I = np.eye(q.shape[0])

    while np.linalg.norm(xd)>0.1:
        ee6=get_ee(ros_interface.latest_joint_states)
        xd=target-ee6 

        J = robot.getJacobian('camera_link')
        
        qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd

        cmd_msg = JointCommand()
        cmd_msg.names = robot.jointNames
        cmd_msg.mode = cmd_msg.VELOCITY_MODE
        cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()

        cmd_msg.velocity = list(qd)
        cmd_pub.publish(cmd_msg)
        time.sleep(delay)

def goto_jointpos(target_jointpos, alpha=0.7, tol=0.1,  delay=0.1):
    cpos=np.array(ros_interface.latest_joint_states.position)
    dpos=np.linalg.norm(target_jointpos-cpos)

    while dpos>tol: 
        cpos=np.array(ros_interface.latest_joint_states.position)
        dpos=np.linalg.norm(target_jointpos-cpos)
 
        qd =(target_jointpos-cpos)*alpha

        cmd_msg = JointCommand()
        cmd_msg.names = robot.jointNames
        cmd_msg.mode = cmd_msg.VELOCITY_MODE
        cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
        cmd_msg.velocity = list(qd)
        cmd_pub.publish(cmd_msg)
        time.sleep(delay)

In [9]:
pos_0=jpos[0] 
npos=[0]+list(pos_0)+[0.0,0.0]
pos_0=np.array(npos)
pos_0

array([ 0.        ,  0.57870605,  0.77856543, -2.17554199,  1.21911426,
       -1.94981445, -1.31956836,  3.0271875 ,  0.        ,  0.        ])

In [10]:
npos=[0]+list(jpos[-1] )+[0.0,0.0]
pos_1=np.array(npos)
pos_1

array([ 0.        ,  0.45991113,  0.81888672, -1.922625  ,  1.37346875,
       -1.99509375, -0.8909502 ,  3.0271875 ,  0.        ,  0.        ])

In [13]:
goto_jointpos(pos_0, alpha=0.7, tol=0.05,  delay=0.1)

In [12]:
goto_jointpos(pos_1, alpha=0.7, tol=0.05,  delay=0.1)

In [14]:
for pos in jpos:
    npos=[0]+list(pos)+[0.0,0.0]
    pos=np.array(npos)
    goto_jointpos(pos, alpha=0.7, tol=0.05,  delay=0.1)

In [15]:
cmd_msg = JointCommand()
cmd_msg.names = robot.jointNames
cmd_msg.mode = cmd_msg.VELOCITY_MODE
cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
cmd_msg.velocity =[0.0]*10
cmd_pub.publish(cmd_msg)